In [ ]:
# API WoS v.1 (unofficial https://github.com/enricobacis/wos/tree/master)

from wos import WosClient
import wos.utils

with WosClient(SID='f312e2fe4d304f467ec3bd7d2af46ea75c5817d7') as client:
    print(wos.utils.query(client, 'AU=Knuth Donald'))

In [ ]:
# API WoS v.2 (official https://github.com/clarivate/wosstarter_python_client)
# Package outdated (setup.py)

import time
import clarivate.wos_starter.client
from pprint import pprint
from clarivate.wos_starter.client.apis.tags import documents_api
from clarivate.wos_starter.client.model.document import Document
from clarivate.wos_starter.client.model.documents_list import DocumentsList

In [54]:
# Homemade WoS API (scratch scrapping)


# If the request is blocked by Clavirate,
# this code might be useful to change the user agent 

# from fake_useragent import UserAgent
# ua = UserAgent()
# headers = {'User-Agent': ua.random}
# response = requests.get(url, headers=headers)

import os
import shutil
import time
import glob
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By


def get_papers_metadata(plateform: int, query: str):
    
    """Get metadata (title, authors, DOI, etc.) of papers coming from scientific literature platforms

    This function provides metadata of scientific articles coming from the
    results of a searching query on scientific literature platform. The
    metadata collected for each article are:
    * DOI
    * Title
    * Authors
    * Publication year
    * Abstract
    * Keywords
    * Language
    * Affiliation
    * Journal
    * Publisher
    * Volume
    * Issue
    The platforms supported are:
    * Web of Science (https://www.webofscience.com/wos/woscc/advanced-search)

    Parameters:
    plateform (int): platform identification code
                     0: Web of Science
    query (str): searching query (must respect the sythax of the platform used)

    Returns:
    xxx

    """

    match plateform:

        case 0:
            
            url = "https://www.webofscience.com/wos/woscc/advanced-search"
            driver = webdriver.Firefox() # manage various web browser
            driver.get(url)

            time.sleep(3)
            driver.find_element(By.ID, "onetrust-reject-all-handler").click()
            
            time.sleep(2)
            search_input = driver.find_element(By.ID, "advancedSearchInputArea")
            search_input.send_keys(query)
            search_input.submit()
            
            time.sleep(2)
            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[1]/div/app-export-option/div/app-export-menu/div/button").click()
            
            time.sleep(1)
            driver.find_element(By.ID, "exportToExcelButton").click()

            records_from_button = driver.find_element(By.ID, "radio3-input")
            driver.execute_script("arguments[0].click();", records_from_button)

            low_bound = driver.find_element(By.ID, "mat-input-1")
            low_bound.clear()
            low_bound.send_keys("2")
            high_bound = driver.find_element(By.ID, "mat-input-2")
            high_bound.clear()
            high_bound.send_keys("1001")

            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button").click()
            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/div/div/div/div[3]").click()
            driver.find_element(By.XPATH, "/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]").click()
            
            time.sleep(10)
            downloaded_files = os.listdir("/home/er/Downloads")
            latest_file = max(downloaded_files, key=os.path.getctime)
            shutil.move(latest_file, "./data/papers_metadata_") #+str(i)


        # case 1:
        #     xxx

        # case _:
        #     # manage error
    
get_papers_metadata(0, "ts = (“meta*analysis” OR “systematic review”) AND (“soil organic carbon” OR SOC OR “soil organic matter” OR SOM OR “soil carbon”)")


ValueError: max() arg is an empty sequence

In [71]:
downloaded_files = os.listdir("/home/er/Downloads")
# print(downloaded_files)
# print(str(os.path.getctime))
latest_file = max(downloaded_files, key=os.path.getctime)
# shutil.move("/home/er/Downloads"+latest_file, "./data/papers_metadata_") #+str(i)

FileNotFoundError: [Errno 2] No such file or directory: 'savedrecs(2).xls'

In [ ]:
# API Sci-Hub (https://github.com/zaytoun/scihub.py/tree/master)

from scihub import SciHub

sh = SciHub()

# exactly the same thing as fetch except downloads the articles to disk
# if no path given, a unique name will be used as the file name
result = sh.download('https://onlinelibrary.wiley.com/doi/full/10.1111/gcb.12569', path='paper.pdf')